# 1. Некоторые операции с классами и объектами

###### 1.1 Напишите код, который выведет на экране все имена полей объекта произвольного пользовательского класса, кроме служебных имен.

In [3]:
class str2(str): ...

In [4]:
print([i for i in dir(str2) if not i.startswith("_")])

['capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'removeprefix', 'removesuffix', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


###### 1.2 Напишите код, который по имени метода, заданному строкой, вызовет этот метод в объекте некоторого пользовательского класса.

In [5]:
str2_val = str2("method call test")
str2_method = "split"

print(getattr(str2_val, str2_method)())

['method', 'call', 'test']


###### 1.3 С кодом ниже что-то не так. Что именно неправильно и как это исправить?

```python
class A:
    pass

class B(A):
    pass

class C(A, B):
    pass
```

In [6]:
# класс С пытается отнаследоваться от B и A, но B уже отнаследовался от A


class A:
    pass


class B(A):
    pass


class C(B):  # C теперь наследуется только от B
    pass

###### 1.4 Напишите функцию-однострочник get_inheritance для вывода строки, отражающей иерархию наследования для входного класса.

In [10]:
def get_inheritance(cls):
    return " --> ".join([c.__name__ for c in cls.__mro__])


print(get_inheritance(C))
print(get_inheritance(str2))
print(get_inheritance(OSError))

C --> B --> A --> object
str2 --> str --> object
OSError --> Exception --> BaseException --> object


# 2. Своя реализация структуры данных

Реализуйте хэш-таблицу, аналог встроенного dict. Используйте для внутренней реализации список пар ключ-значение. Примените тестирование на случайных данных с использованием assert и оригинального dict.

1. Реализуйте методы чтения, записи и получения размера хэш-таблицы.
2. Реализуйте для методов своей хэш-таблицы тот же интерфейс, что и в dict, включая перегрузку операций.
3. Реализуйте поддержку итератора для цикла for. Обязательно протестируйте код на примерах с вложенными циклами!

In [51]:
class HashMap:
    """Simple hash map implementation with linear probing for collisions."""

    def __init__(self, capacity=100) -> None:
        """A hash map is initialized with a fixed capacity (default is 100)."""
        self.capacity = capacity
        self.size = 0
        self.buckets = [[] for _ in range(capacity)]
        self.iter_index = 0
        self.iter_bucket_index = 0

    def _hash(self, key) -> int:
        """Типа хэш-функция, которая просто возвращает остаток от деления на capacity."""
        return hash(key) % self.capacity

    def _find_bucket_and_index(self, key) -> tuple:
        """Поиск бакета и индекса в бакете для ключа."""
        hash_key = self._hash(key)
        bucket = self.buckets[hash_key]
        for i, (k, _) in enumerate(bucket):
            if k == key:
                return bucket, i
        return bucket, None

    def __setitem__(self, key, value) -> None:
        """Установка значения по ключу. Если ключ уже существует, то его значение обновляется."""
        bucket, index = self._find_bucket_and_index(key)
        if index is not None:
            bucket[index] = (key, value)
        else:
            bucket.append((key, value))
            self.size += 1

    def __getitem__(self, key) -> any:
        """Получение значения по ключу. Если ключ не найден, то райзится исключение KeyError."""
        bucket, index = self._find_bucket_and_index(key)
        if index is not None:
            return bucket[index][1]
        raise KeyError("Key not found: {}".format(key))

    def __delitem__(self, key) -> None:
        """Удаление значения по ключу. Если ключ не найден, то райзится исключение KeyError."""
        bucket, index = self._find_bucket_and_index(key)
        if index is not None:
            del bucket[index]
            self.size -= 1
        else:
            raise KeyError("Key not found: {}".format(key))

    def __len__(self) -> int:
        """Возвращает количество элементов в хэш-таблице."""
        return self.size

    def __iter__(self) -> "HashMap":
        """Итератор по ключам."""
        self.iter_index = 0
        self.iter_bucket_index = 0
        return self

    def __next__(self) -> any:
        """Получение следующего ключа."""
        while self.iter_index < self.capacity:
            if self.iter_bucket_index < len(self.buckets[self.iter_index]):
                key = self.buckets[self.iter_index][self.iter_bucket_index][0]
                self.iter_bucket_index += 1
                return key
            else:
                self.iter_index += 1
                self.iter_bucket_index = 0
        raise StopIteration

    def __repr__(self) -> str:
        """Представление хэш-таблицы в виде строки."""
        items = [f"{repr(k)}: {repr(v)}" for bucket in self.buckets for k, v in bucket]
        return "{\n  " + ",\n  ".join(items) + "\n}"

In [52]:
hm = HashMap()

In [53]:
hm["key1"] = "value1"
hm["key2"] = "value2"
hm["key3"] = "value3"

In [54]:
for key in hm:
    print(f"{key}: {hm[key]}")

key2: value2
key3: value3
key1: value1


In [55]:
print(len(hm))

3


In [56]:
hm["key1"] = "new value1"

In [57]:
hm

{
  'key2': 'value2',
  'key3': 'value3',
  'key1': 'new value1'
}

In [58]:
del hm["key2"]

In [59]:
hm

{
  'key3': 'value3',
  'key1': 'new value1'
}

Сделаем теперь тесты через `assert`

In [60]:
import random
import string


def random_string(length=10) -> str:
    """Генерирует случайную строку заданной длины."""
    letters = string.ascii_lowercase
    return "".join(random.choice(letters) for i in range(length))

In [62]:
# простой набор рандомных данных
keys = [random_string(5) for _ in range(50)]
values = [random.randint(1, 100) for _ in range(50)]

my_map = HashMap()
reference_dict = dict()

In [63]:
# Заполнение структур данными и проверка соответствия значений
for key, value in zip(keys, values):
    my_map[key] = value
    reference_dict[key] = value
    assert my_map[key] == reference_dict[key], "Mismatched values on insertion"

In [64]:
# Проверка наличия ключей в структуре
for key in keys:
    assert key in my_map, f"Key {key} not found in SimpleHashMap"
    assert my_map[key] == reference_dict[key], "Mismatched values on retrieval"

In [65]:
# Проверка удаления ключей
for key in keys[:10]:  # Удаляем первые 10 элементов
    del my_map[key]
    del reference_dict[key]

for key in keys[10:]:  # Проверяем оставшиеся ключи
    assert my_map[key] == reference_dict[key], "Mismatch after deletion"

Как видим, все тесты проходят. Задание выполнено успешно. Ставим 1000 баллов за задание.

# 3. Деревья выражений

Требуется реализовать операции для печати, вычисления арифметических выражений и компиляции выражений в код стековой машины.

###### 3.1 Реализовать классы узлов дерева: `Num`, `Add` и `Mul`. Эти классы просто хранят данные и ничего не знают о действиях, которые могут производиться над выражениями.

Пример, который будет использоваться далее:

```python
ast = Add(Num(7), Mul(Num(3), Num(2)))
```

In [75]:
class Num:
    def __init__(self, value):
        self.value = value


class Add:
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Mul:
    def __init__(self, left, right):
        self.left = left
        self.right = right

In [76]:
ast = Add(Num(7), Mul(Num(3), Num(2)))

###### 3.2. Реализовать класс-посетитель PrintVisitor для печати выражения. Обойтись без перегрузки repr и str, а также без операторов ветвления!

Пример:
```python
>>> pv = PrintVisitor()
>>> print(pv.visit(ast))
(7 + (3 * 2))
```

In [77]:
from typing import NoReturn


class PrintVisitor:
    """Visitor для печати AST."""

    def visit(self, node) -> str:
        # Динамически определяем метод на основе типа узла по названию и вызываем его
        method_name = f"visit_{type(node).__name__}"
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def visit_Num(self, node) -> str:
        return str(node.value)

    def visit_Add(self, node) -> str:
        return f"({self.visit(node.left)} + {self.visit(node.right)})"

    def visit_Mul(self, node) -> str:
        return f"({self.visit(node.left)} * {self.visit(node.right)})"

    def generic_visit(self, node) -> NoReturn:
        raise Exception(f"No visit_{type(node).__name__} method")

In [78]:
pv = PrintVisitor()
print(pv.visit(ast))

(7 + (3 * 2))


###### 3.3. Реализовать класс-посетитель CalcVisitor для вычисления значения выражения. Обойтись без операторов ветвления!


Пример:
```python
>>> cv = CalcVisitor()
>>> print(cv.visit(ast))
13
```

In [79]:
class CalcVisitor:
    """Вычисление значения арифметического выражения."""

    def visit(self, node) -> int:
        # Аналогично, делегирование на соответствующий типу узла метод
        method_name = "visit_" + type(node).__name__
        method = getattr(self, method_name, self.generic_visit)
        return method(node)

    def visit_Num(self, node) -> int:
        return node.value

    def visit_Add(self, node) -> int:
        return self.visit(node.left) + self.visit(node.right)

    def visit_Mul(self, node) -> int:
        return self.visit(node.left) * self.visit(node.right)

    def generic_visit(self, node) -> NoReturn:
        raise Exception(f"No visit_{type(node).__name__} method")

In [80]:
cv = CalcVisitor()
print(cv.visit(ast))

13


###### 3.4. Реализовать класс-посетитель StackVisitor для компиляции выражения в код стековой машины. Обойтись без операторов ветвления!

Пример:
```python
>>> sv = StackVisitor()
>>> print(sv.visit(ast))
PUSH 7
PUSH 3
PUSH 2
MUL
ADD
```

In [81]:
class StackVisitor:
    def visit(self, node):
        method_name = "visit_" + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def visit_Num(self, node) -> str:
        return f"PUSH {node.value}"

    def visit_Add(self, node) -> str:
        return f"{self.visit(node.left)}\n{self.visit(node.right)}\nADD"

    def visit_Mul(self, node) -> str:
        return f"{self.visit(node.left)}\n{self.visit(node.right)}\nMUL"

    def generic_visit(self, node) -> NoReturn:
        raise Exception(f"No visit_{type(node).__name__} method")

In [82]:
sv = StackVisitor()
print(sv.visit(ast))

PUSH 7
PUSH 3
PUSH 2
MUL
ADD


# 4. Предметно-ориентированный язык HTML-тегов

###### 4.1. (уровень сложности: средний)

Реализовать язык HTML-тегов с помощью менеджера контекста.

Реализовать классы для выполнения следующего примера:
```python
html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')
```

С помощью `html.get_code()` выдается следующий результат:

```html
<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>
```

In [83]:
from contextlib import contextmanager

In [100]:
class HTML:

    def __init__(self):
        self.intend = 0
        self.elements = list()

    @contextmanager
    def body(self):
        self.elements.append(f"{self.intend*" "}<body>")
        self.intend += 2
        yield
        self.intend -= 2
        self.elements.append(f"{self.intend*" "}</body>")

    @contextmanager
    def div(self):
        self.elements.append(f"{self.intend*" "}<div>")
        self.intend += 2
        yield
        self.intend -= 2
        self.elements.append(f"{self.intend*" "}</div>")

    def p(self, line):
        self.elements.append(f"{self.intend*" "}<p>{line}</p>")

    def get_code(self):
        return "\n".join(self.elements)

In [101]:
html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p("Первая строка.")
            html.p("Вторая строка.")
        with html.div():
            html.p("Третья строка.")

print(html.get_code())

<body>
  <div>
    <div>
      <p>Первая строка.</p>
      <p>Вторая строка.</p>
    </div>
    <div>
      <p>Третья строка.</p>
    </div>
  </div>
</body>


# 5. Визуализация графов

###### 5.1. (уровень сложности: средний)

Реализовать прототип библиотеки для рисования в формате SVG.

Для некоторых из следующих задач потребуется библиотека для работы с SVG-графикой. Достаточно поддержать только рисование линий и кругов с указанием цвета.

Пример:

```python
svg = SVG()

svg.line(10, 10, 60, 10, color='black')
svg.line(60, 10, 60, 60, color='black')
svg.line(60, 60, 10, 60, color='black')
svg.line(10, 60, 10, 10, color='black')

svg.circle(10, 10, r=5, color='red')
svg.circle(60, 10, r=5, color='red')
svg.circle(60, 60, r=5, color='red')
svg.circle(10, 60, r=5, color='red')

svg.save('pic.svg', 100, 100)
```

Файл `pic.svg` будет иметь следующий вид:

```xml
<svg version="1.1" width="100.000000" height="100.000000" xmlns="http://www.w3.org/2000/svg">
<line x1="10.000000" y1="10.000000" x2="60.000000" y2="10.000000" stroke="black" />
<line x1="60.000000" y1="10.000000" x2="60.000000" y2="60.000000" stroke="black" />
<line x1="60.000000" y1="60.000000" x2="10.000000" y2="60.000000" stroke="black" />
<line x1="10.000000" y1="60.000000" x2="10.000000" y2="10.000000" stroke="black" />
<circle cx="10.000000" cy="10.000000" r="5.000000" fill="red" />
<circle cx="60.000000" cy="10.000000" r="5.000000" fill="red" />
<circle cx="60.000000" cy="60.000000" r="5.000000" fill="red" />
<circle cx="10.000000" cy="60.000000" r="5.000000" fill="red" />
</svg>
```

In [104]:
class SVG:
    def __init__(self):
        self.elements = []

    def line(self, x1: float, y1: float, x2: float, y2: float, color: str = "black"):
        """Добавление линии."""
        self.elements.append(
            f'<line x1="{x1:.6f}" y1="{y1:.6f}" x2="{x2:.6f}" y2="{y2:.6f}" stroke="{color}" />'
        )

    def circle(self, cx: float, cy: float, r: float, color: str = "black"):
        """Добавление круга."""
        self.elements.append(
            f'<circle cx="{cx:.6f}" cy="{cy:.6f}" r="{r:.6f}" fill="{color}" />'
        )

    def save(self, filename: str, width: float, height: float):
        """Сохранение SVG-файла."""
        with open(filename, "w") as file:
            file.write('<svg version="1.1" ')
            file.write(f'width="{width:.6f}" height="{height:.6f}" ')
            file.write('xmlns="http://www.w3.org/2000/svg">\n')
            for element in self.elements:
                file.write(element + "\n")
            file.write("</svg>")

In [105]:
svg = SVG()

svg.line(10, 10, 60, 10, color="black")
svg.line(60, 10, 60, 60, color="black")
svg.line(60, 60, 10, 60, color="black")
svg.line(10, 60, 10, 10, color="black")

svg.circle(10, 10, r=5, color="red")
svg.circle(60, 10, r=5, color="red")
svg.circle(60, 60, r=5, color="red")
svg.circle(10, 60, r=5, color="red")

svg.save("pic.svg", 100, 100)

Посмотрим результат:

![pic.svg](pic.svg)

Как видим, все работает. Задание выполнено успешно. Ставим 1000 баллов за задание.

###### 5.2. уровень сложности: средний

Реализуйте простой алгоритм Д. Кнута для визуализации бинарных деревьев. Вся работа этого и последующих алгоритмов визуализации графов заключается в назначении координат (x, y) каждому узлу.

Основная идея алгоритма состоит в том, чтобы нарисовать сначала левое поддерево, а затем, правее этого поддерева нарисовать текущий узел и, наконец, правее этого текущего узла нарисовать правое поддерево. Каждый следующий уровень в дереве увеличивает координату y.

В наивной реализации текущая позиция по x задается глобальной переменной, избавьтесь от нее с помощью класса.

Вспомогательный код для решения:

```python
scale_x = 25
scale_y = 50

class Tree:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.x = 0
        self.y = 0

```

Пример визуализации для следующего определения дерева:

```python
tree_2 = Tree(2, Tree(3, Tree(4), Tree(5)), Tree(6, Tree(7)))
tree_8 = Tree(8, Tree(9, Tree(10), Tree(11, Tree(12), Tree(13))), Tree(14))
tree = Tree(1, tree_2, tree_8)
```

![true-grue/kispython/knuth_tree](https://raw.githubusercontent.com/true-grue/kispython/33f75d6358336498fef8c6c7138bc72933ba499f/data/knuth_tree.png)

In [130]:
class Tree:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.x = 0
        self.y = 0

In [131]:
from typing import Union

TreeOrNone = Union[Tree, None]


class TreeDrawer:
    def __init__(self, scale_x: int = 25, scale_y: int = 50):
        self.scale_x = scale_x
        self.scale_y = scale_y
        self.next_x = 0

    def layout_tree(self, tree: TreeOrNone, depth=0):
        """Расположение узлов дерева"""
        if tree is not None:
            # отсчитываем координаты для левого поддерева
            self.layout_tree(tree.left, depth + 1)
            # устанавливаем координаты узла
            tree.x = self.next_x
            tree.y = depth
            # инкрементим счетчик для следующего узла
            self.next_x += 1
            self.layout_tree(tree.right, depth + 1)

    def visualize_tree(self, tree: TreeOrNone) -> SVG:
        """Визуализация дерева."""
        # располагаем узлы
        self.layout_tree(tree)
        # создаем SVG
        svg = SVG()
        self.draw_tree(svg, tree)
        return svg

    def draw_tree(self, svg, tree: TreeOrNone, parent_coords=None):
        """Рекурсивная функция для отрисовки дерева."""
        if tree is not None:
            # считаем реальные координаты узла
            x = tree.x * self.scale_x
            y = tree.y * self.scale_y
            # рисуем узел
            svg.circle(x, y, r=5, color="black")
            # рисуем линию к родителю
            if parent_coords is not None:
                svg.line(parent_coords[0], parent_coords[1], x, y, color="black")
            # рекурсивно рисуем левое и правое поддерево
            self.draw_tree(svg, tree.left, (x, y))
            self.draw_tree(svg, tree.right, (x, y))

In [134]:
tree_2 = Tree(2, Tree(3, Tree(4), Tree(5)), Tree(6, Tree(7)))
tree_8 = Tree(8, Tree(9, Tree(10), Tree(11, Tree(12), Tree(13))), Tree(14))
tree = Tree(1, tree_2, tree_8)

visualizer = TreeDrawer()
svg = visualizer.visualize_tree(tree)
svg.save("tree_visualization.svg", 300, 300)

Результат:

![tree_visualization.svg](tree_visualization.svg)

Как видим, все работает. Задание выполнено успешно. Ставим 1000 баллов за задание.

<details> Спойлер по поводу 8.2
Ученый уровень постараюсь сделать и отправить вам в ближайшее время :) или показать на занятии
</details>

In [135]:
!tar -czf prac4_distant_Абраамян_АВ.tar.gz pic.svg tree_visualization.svg prac.ipynb